In [ ]:
#written by Arezoo Borji
# import numpy as np
# import pandas as pd
# import os
# for dirname, _, filenames in os.walk('/content/drive/MyDrive/alzheimer_Dataset1'):
#     for filename in filenames[0:1000]:
#         print(os.path.join(dirname, filename))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime
import random
import nibabel as nib
import glob
import os

# from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_curve,auc
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,optimizers
import cv2
import imageio
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="1newkfoldtransferlearningADvsCN2")

config = wandb.config
config.learning_rate = 0.001

In [ ]:
dataset_path = '/content/drive/MyDrive/'
width = height = 224
batch_size = 64
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = False,
    # zoom_range = 0.1,
    validation_split = 0.15


In [ ]:
train_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'alzheimer_dataset1'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
    subset= 'training'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
val_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'alzheimer_dataset1'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
    subset= 'validation'
)

In [ ]:
test_data = data_generator.flow_from_directory(
    os.path.join(dataset_path,'/content/drive/MyDrive/test1'),
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
)

In [ ]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape = (width,height,3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
)

In [ ]:
for layer in base_model.layers[:-8]:
  layer.trainable = False

In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize=(8,8))

for i in range(16):
  plt.subplot(4,4,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(train_images[i],cmap=plt.cm.binary)

In [ ]:
import keras
from keras import layers
from keras import models
import tensorflow as tf
from keras.layers import Dropout
layers=[

    # tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    # tf.keras.layers.MaxPooling2D(2,2),

    # tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    # tf.keras.layers.MaxPooling2D(2,2),

    # tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    # tf.keras.layers.MaxPooling2D(2,2),
    # tf.keras.layers.Dropout(0.4),

    # tf.keras.layers.Flatten(),

     # 512 neuron hidden layer
     base_model,
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(2, activation="sigmoid")

]

In [ ]:

model=tf.keras.Sequential(layers)
model.compile(optimizers.Adam(learning_rate=config.learning_rate),
             loss='binary_crossentropy',
             metrics = ['accuracy'] )

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_data,
                              validation_data=val_data,
                              epochs=10,
                    callbacks = [WandbCallback()])

In [ ]:
import matplotlib.pyplot as plt
 accuracy = history.history['accuracy']
 val_accuracy = history.history['val_accuracy']
 loss = history.history['loss']
 val_loss = history.history['val_loss']
 epochs = range(len(accuracy))
 plt.plot(epochs, accuracy, 'r', label="Training accuracy")
 plt.plot(epochs, val_accuracy, 'b', label="Validation accuracy")
 plt.title("Training and test accuracy")
 plt.legend()
 plt.figure()
 plt.plot(epochs, loss, 'r', label="Training loss")
 plt.plot(epochs, val_loss, 'b', label="Training Validation_loss")
 plt.title("Training and test loss")
 plt.legend()
 plt.figure()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from keras.models import Sequential
from keras.layers import Activation, Dense,Flatten,LSTM,Dropout
import tensorflow as tf
import numpy as np

# Define per-fold score containers <-- these are new
acc_per_fold = []
loss_per_fold = []


# Merge inputs and targets
inputs = np.concatenate((val_data,test_data),axis=0)
targets = np.concatenate((val_data,test_data),axis=0)

# Define the K-fold Cross Validator
kfold = KFold(5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
  # create and fit resnet
  model = Sequential()
  layers=[base_model,
      tf.keras.layers.Dense(512, activation="relu"),
      tf.keras.layers.Dense(2, activation="sigmoid")
  ]
  model.compile(optimizers.Adam(learning_rate=config.learning_rate),
             loss='binary_crossentropy',
             metrics = ['accuracy'] )

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  # Fit data to model
  history = model.fit(train_data,
                      validation_data=val_data,
                      epochs=8,
                      callbacks = [WandbCallback()])

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1